<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

In [ ]:
!pip install tensorflow

In [ ]:
!pip install qdrant-client

#รวม

1. เอาภาพเข้ามา
2. ครอปภาพ
3. เอาภาพไปหาความเหมือนในqdrant
4. เอาผลค่าความเหมือนมาเข้าเงื่อนไข
5. แสดงผลลัพท์ภาพต้นฉบับ+ชื่อคลาสที่ทำนายได้

In [ ]:
import os
from PIL import Image
import numpy as np
from ultralytics import YOLO
from qdrant_client import QdrantClient
from transformers import CLIPProcessor, CLIPModel
import torch
import matplotlib.pyplot as plt

# Load your YOLOv8 model
model = YOLO('/content/drive/MyDrive/drug/best_new.pt')

# Define the input image path
input_image_path = '/content/Oseni25_15mg-n.jpg'

# Connect to Qdrant
qdrant_client = QdrantClient(
    url="https://8366dca9-9b40-481c-9a7c-102b62b118c2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Ft3t6L99uoTsJW8IOk5VH2byKU-BQYqiieuxXFBDp99wo75od0ddAw",
)
collection_name = "vector_CLIP"

# Load CLIPModel and processor for embedding generation
clip_model_name = "openai/clip-vit-large-patch14"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

# Function to generate image embedding
def image_embedding(image):
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = clip_model.get_image_features(**inputs)
    return outputs[0].cpu().numpy()

# Function to search for similar items in Qdrant
def search_similar_items(query_image, client, collection_name):
    query_embedding = image_embedding(query_image)
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=2  # Get top 2 results
    )

    if len(search_result) < 2:
        print("Not enough results found.")
        return None, None

    return search_result[0], search_result[1]

# Function to process detection and cropping
def detect_and_crop(image_path):
    results = model(image_path)
    for result in results:
        boxes = result.boxes.xyxy.cpu().detach().numpy()
        labels = result.boxes.cls.cpu().detach().numpy()
        scores = result.boxes.conf.cpu().detach().numpy()

        threshold = 0.3
        for i, label in enumerate(labels):
            if scores[i] >= threshold:
                x1, y1, x2, y2 = map(int, boxes[i])
                im = Image.open(image_path).convert('RGB')
                im_crop = im.crop((x1, y1, x2, y2))

                return im_crop

    print(f"No target objects detected in '{image_path}'.")
    return None

# Function to apply conditions, display output, and show cropped image with similarity score
def apply_conditions_and_display(image_path):
    cropped_image = detect_and_crop(image_path)
    if cropped_image is None:
        return

    top_1, top_2 = search_similar_items(cropped_image, qdrant_client, collection_name)
    if top_1 is None or top_2 is None:
        return

    score_1 = top_1.score
    score_2 = top_2.score
    class_name = top_1.payload.get('class', 'Unknown')

    # Condition 1: Score > 0.9 and difference between top 1 and top 2 is > 2%
    if score_1 > 0.9:
        if (score_1 - score_2) >= 0.02:
            prediction = f"Class Name: {class_name}, Score 1: {score_1:.4f}, Score 2: {score_2:.4f}"
        else:
            prediction = f"Top 1 and top 2 scores are close (Score 1: {score_1:.4f}, Score 2: {score_2:.4f}). Please retake the image."

    # Condition 2: 0.85 < score < 0.9
    elif 0.85 < score_1 < 0.9:
        prediction = f"This drug might be in the class. Score 1: {score_1:.4f}. Please retake the image."

    # Condition 3: score < 0.85
    else:
        prediction = f"This drug is not in the class. Score 1: {score_1:.4f}. Saving as a new class."
        # Add the logic for saving as a new class here

    # Display the cropped image with similarity score
    plt.figure(figsize=(8, 8))
    plt.imshow(cropped_image)
    plt.axis('off')
    plt.title(prediction)
    plt.show()

# Process the single image
apply_conditions_and_display(input_image_path)

In [ ]:
# แทน
import os
from PIL import Image
import numpy as np
from ultralytics import YOLO
from qdrant_client import QdrantClient
from transformers import CLIPProcessor, CLIPModel
import torch
import matplotlib.pyplot as plt

# Load your YOLOv8 model
model = YOLO('/content/drive/MyDrive/drug/best_new.pt')

# Define the input image path
input_image_path = '/content/Oseni25_15mg-n.jpg'

# Connect to Qdrant
qdrant_client = QdrantClient(
    url = "https://a63ffbf5-5568-46dd-9ec3-98751a51a998.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key = "S0QgrdtYHTC8f_53Nes2uJ4gWoxbPnIwkujhfRlwcWA_MOvuGseLXw"
)
collection_name = "medicine50classClipModel"

# Load CLIPModel and processor for embedding generation
clip_model_name = "openai/clip-vit-large-patch14"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

# Function to generate image embedding
def image_embedding(image):
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = clip_model.get_image_features(**inputs)
    return outputs[0].cpu().numpy()

# Function to search for similar items in Qdrant
def search_similar_items(query_image, client, collection_name):
    query_embedding = image_embedding(query_image)
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=2  # Get top 2 results
    )

    if len(search_result) < 2:
        print("Not enough results found.")
        return None, None

    return search_result[0], search_result[1]

# Function to process detection and cropping
def detect_and_crop(image_path):
    results = model(image_path)
    for result in results:
        boxes = result.boxes.xyxy.cpu().detach().numpy()
        labels = result.boxes.cls.cpu().detach().numpy()
        scores = result.boxes.conf.cpu().detach().numpy()

        threshold = 0.3
        for i, label in enumerate(labels):
            if scores[i] >= threshold:
                x1, y1, x2, y2 = map(int, boxes[i])
                im = Image.open(image_path).convert('RGB')
                im_crop = im.crop((x1, y1, x2, y2))

                return im_crop

    print(f"No target objects detected in '{image_path}'.")
    return None

# Function to apply conditions, display output, and show cropped image with similarity score
def apply_conditions_and_display(image_path):
    cropped_image = detect_and_crop(image_path)
    if cropped_image is None:
        return

    top_1, top_2 = search_similar_items(cropped_image, qdrant_client, collection_name)
    if top_1 is None or top_2 is None:
        return

    score_1 = top_1.score
    score_2 = top_2.score
    class_name = top_1.payload.get('class', 'Unknown')

    # Condition 1: Score > 0.9 and difference between top 1 and top 2 is > 2%
    if score_1 > 0.9:
        if (score_1 - score_2) >= 0.02:
            prediction = f"Class Name: {class_name}, Score 1: {score_1:.4f}, Score 2: {score_2:.4f}"
        else:
            prediction = f"Top 1 and top 2 scores are close (Score 1: {score_1:.4f}, Score 2: {score_2:.4f}). Please retake the image."

    # Condition 2: 0.85 < score < 0.9
    elif 0.85 < score_1 < 0.9:
        prediction = f"This drug might be in the class. Score 1: {score_1:.4f}. Please retake the image."

    # Condition 3: score < 0.85
    else:
        prediction = f"This drug is not in the class. Score 1: {score_1:.4f}. Saving as a new class."
        # Add the logic for saving as a new class here

    # Display the cropped image with similarity score
    plt.figure(figsize=(8, 8))
    plt.imshow(cropped_image)
    plt.axis('off')
    plt.title(prediction)
    plt.show()

# Process the single image
apply_conditions_and_display(input_image_path)

In [ ]:
import os
import json
import torch
import numpy as np
from PIL import Image
import streamlit as st
from ultralytics import YOLO
from qdrant_client import QdrantClient
from transformers import CLIPProcessor, CLIPModel

# Load your YOLOv8 model
model = YOLO('model.pt')

# Connect to Qdrant
qdrant_client = QdrantClient(
    url = "https://a63ffbf5-5568-46dd-9ec3-98751a51a998.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key = "S0QgrdtYHTC8f_53Nes2uJ4gWoxbPnIwkujhfRlwcWA_MOvuGseLXw"
)
collection_name = "medicine50classClipModel"

# Load CLIPModel and processor for embedding generation
clip_model_name = "openai/clip-vit-large-patch14"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

# Function to generate image embedding
def image_embedding(image):
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = clip_model.get_image_features(**inputs)
    return outputs[0].cpu().numpy()

# Function to search for similar items in Qdrant
def search_similar_items(query_image, client, collection_name):
    query_embedding = image_embedding(query_image)
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=2  # Get top 2 results
    )

    if len(search_result) < 2:
        st.write("Not enough results found.")
        return None, None

    return search_result[0], search_result[1]

# Function to process detection and cropping
def detect_and_crop(image_file):
    # Load image using PIL from file-like object
    image = Image.open(image_file).convert('RGB')

    results = model(image)
    for result in results:
        boxes = result.boxes.xyxy.cpu().detach().numpy()
        labels = result.boxes.cls.cpu().detach().numpy()
        scores = result.boxes.conf.cpu().detach().numpy()

        threshold = 0.3
        for i, score in enumerate(scores):
            if score >= threshold:
                x1, y1, x2, y2 = map(int, boxes[i])
                im_crop = image.crop((x1, y1, x2, y2))
                return im_crop

    st.write("No target objects detected.")
    return None

# Streamlit UI
st.title('Drug Identification and Classification')

uploaded_file = st.file_uploader("Upload an image", type=['jpg', 'png', 'jpeg'])

if uploaded_file is not None:
    st.image(uploaded_file, caption='Uploaded Image', use_column_width=True)

    # Process image
    cropped_image = detect_and_crop(uploaded_file)

    if cropped_image:
        top_1, top_2 = search_similar_items(cropped_image, qdrant_client, collection_name)
        if top_1 and top_2:
            score_1 = top_1.score
            score_2 = top_2.score
            class_name = top_1.payload.get('class', 'Unknown')

            # Apply conditions
            if score_1 > 0.9:
                if (score_1 - score_2) >= 0.02:
                    prediction = f"Class Name: {class_name}, Score 1: {score_1:.4f}, Score 2: {score_2:.4f}"
                else:
                    prediction = f"Top 1 and top 2 scores are close (Score 1: {score_1:.4f}, Score 2: {score_2:.4f}). Please retake the image."

            elif 0.85 < score_1 < 0.9:
                prediction = f"This drug might be in the class. Score 1: {score_1:.4f}. Please retake the image."

            else:
                prediction = f"This drug is not in the class. Score 1: {score_1:.4f}. Saving as a new class."

            st.write(prediction)
            st.image(cropped_image, caption='Cropped Image', use_column_width=True)

In [ ]:
#ใช้อันนี้
import io
import numpy as np
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import cv2
from ultralytics import YOLO
from qdrant_client import QdrantClient
import streamlit as st
import os

# Initialize the models and Qdrant client (using your existing setup)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
yolo_model_path = r"C:\Users\Admin\Documents\INET\Drug\Medicine_StreamlitApp\best_segment.pt"
model_detection = YOLO(yolo_model_path)
model_segmentation = YOLO(yolo_model_path)
qdrant_url = "https://a63ffbf5-5568-46dd-9ec3-98751a51a998.us-east4-0.gcp.cloud.qdrant.io:6333"
api_key = "S0QgrdtYHTC8f_53Nes2uJ4gWoxbPnIwkujhfRlwcWA_MOvuGseLXw"
collection_name = "medicine50classClipModel1.1"
client = QdrantClient(url=qdrant_url, api_key=api_key)

def detect_and_crop_objects(image):
    results = model_detection(image)
    cropped_images = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            cropped_image = image[int(y1):int(y2), int(x1):int(x2)]
            cropped_images.append(cropped_image)
    return cropped_images

def segment_background(image):
    results = model_segmentation.predict(source=image)
    black_image = np.zeros_like(image)
    for result in results:
        if result.masks is not None:
            masks = result.masks.xy
            for mask in masks:
                mask = np.array(mask, dtype=np.int32)
                cv2.fillPoly(black_image, [mask], (255, 255, 255))
    binary_mask = cv2.cvtColor(black_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(binary_mask, 1, 255, cv2.THRESH_BINARY)
    result_image = cv2.bitwise_and(image, image, mask=binary_mask)
    return result_image

def get_image_vector(image):
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).convert("RGB")
    inputs = processor(images=pil_image, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    return image_features.cpu().numpy().flatten()

def find_top_k_similar_classes_with_qdrant(image, unseenmedicine_folder, k=5, top_n=1000):
    image_vector = get_image_vector(image)
    search_result = client.search(
        collection_name=collection_name,
        query_vector=image_vector.tolist(),
        limit=top_n,
        with_payload=True,
        with_vectors=False
    )
    top_results = [(result.payload.get("classname", "unknown"), result.score) for result in search_result]
    unique_classes = {}
    for class_name, score in top_results:
        if class_name not in unique_classes:
            unique_classes[class_name] = score
        if len(unique_classes) == k:
            break
    filtered_top_k_classes = sorted(unique_classes.items(), key=lambda x: x[1], reverse=True)
    if len(filtered_top_k_classes) < k:
        st.warning("Not enough unique classes found.")
        return
    top_1_score = filtered_top_k_classes[0][1]
    top_2_score = filtered_top_k_classes[1][1] if len(filtered_top_k_classes) > 1 else 0
    if top_1_score > 0.9 and (top_1_score - top_2_score) > 0.02:
        st.success(f"Top prediction: {filtered_top_k_classes[0][0]} with score {top_1_score:.4f}")
    elif top_1_score > 0.9 and (top_1_score - top_2_score) <= 0.02:
        st.error("Please take a clearer picture. >_<")
    elif top_1_score <= 0.9 and top_1_score >= 0.85:
        st.error("Please take a clearer picture. -_-")
    elif top_1_score < 0.85:
        st.error("I have never seen this medicine before.")
        if not os.path.exists(unseenmedicine_folder):
            os.makedirs(unseenmedicine_folder)
        # Save the image only if the condition is met
        # shutil.copy(image_path, unseenmedicine_folder)
        st.info(f"Image saved to {unseenmedicine_folder}")

# Streamlit application layout
st.title("Medicine Image Processing")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    # Read the image file directly from memory
    image = Image.open(uploaded_file)
    image = np.array(image.convert("RGB"))

    # Automatically start processing when an image is uploaded
    unseenmedicine_folder = r"C:\Users\Admin\Documents\INET\Drug\Medicine_StreamlitApp\unseenmedicine"
    find_top_k_similar_classes_with_qdrant(image, unseenmedicine_folder, k=5)